In [1]:
import pandas as pd
import numpy as np

In [ ]:
from fredapi import Fred
fred = Fred(api_key='INSERT KEY HERE')

In [3]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
}

In [4]:
# state_gdp = pd.read_csv('SQGDP1__ALL_AREAS_2018_2024.csv')
# cleaned_cols = state_gdp.drop((i for i in range(len(state_gdp)) if state_gdp['GeoName'][i] not in us_state_to_abbrev or state_gdp['Description'][i][:4] != 'Real'))
# cleaned_cols = cleaned_cols.drop(['GeoFIPS', 'TableName', 'Region', 'IndustryClassification', 'LineCode', 'Description', 'Unit'], axis=1)
# cleaned_cols = cleaned_cols.reset_index().drop('index', axis=1)

In [5]:
start_year = 1997
end_year = 2024

### State minimum wage data

In [5]:
fed_minwage = fred.get_series('STTMINWGFG')
fed_minwage_df = pd.DataFrame(fed_minwage, columns=['Minimum wage'])
fed_minwage_transpose = fed_minwage_df.T
state_minimum_wages = pd.DataFrame(columns=fed_minwage_transpose.columns)
for state in us_state_to_abbrev:
    id = us_state_to_abbrev[state]
    try:
        state_minwage_df = pd.DataFrame(fred.get_series('STTMINWG'+id), columns=['Minimum wage'])
        state_minwage_df = np.maximum(state_minwage_df, fed_minwage_df).T.iloc[0]
        state_minimum_wages.loc[state] = state_minwage_df.fillna(fed_minwage_df['Minimum wage'])
        continue
    except ValueError as e:
        state_minimum_wages.loc[state] = fed_minwage_transpose.iloc[0]
        print(f'{state} does not have a minimum wage series')
state_minimum_wages.sort_index(axis=0, inplace=True)
state_minimum_wages.rename(columns=lambda x : str(x)[:4], inplace=True)
state_minimum_wages.drop([col for col in state_minimum_wages.columns if (int(col) < start_year or int(col) >= end_year)], axis=1, inplace=True)
state_minimum_wages.to_csv('min_wages_by_state.csv')

Alabama does not have a minimum wage series
Louisiana does not have a minimum wage series
Mississippi does not have a minimum wage series
South Carolina does not have a minimum wage series
Tennessee does not have a minimum wage series


### State GDP data

In [35]:
state_gdp = pd.DataFrame(columns = pd.DataFrame(fred.get_series('CANGSP')).T.columns)
for state in us_state_to_abbrev:
    id = us_state_to_abbrev[state]
    state_gdp_df = pd.DataFrame(fred.get_series(id+'NGSP'))
    state_gdp.loc[state] = state_gdp_df.T.iloc[0]
state_gdp.sort_index(axis=0, inplace=True)
state_gdp.rename(columns=lambda x : str(x)[:4], inplace=True)
state_gdp.drop([col for col in state_gdp.columns if int(col) < start_year], axis=1, inplace=True)
state_gdp.to_csv('gdp_by_state.csv')

### State unemployment data

In [36]:
years = set(pd.DataFrame(fred.get_series('WAUR')).T.columns.year)
years.remove(2024)

state_unempl = pd.DataFrame(columns = sorted(years))
for state in us_state_to_abbrev:
    id = us_state_to_abbrev[state]
    state_unempl_df = pd.DataFrame(fred.get_series(id+'UR'))
    for yr in years:
        avg = round(sum([state_unempl_df[0][str(yr)+'-'+str(month)+'-01'] for month in range(1, 13)])/12, 2)
        state_unempl.loc[state, yr] = avg
state_unempl.sort_index(axis=0, inplace=True)

state_unempl.drop([col for col in state_unempl.columns if int(col) < start_year], axis=1, inplace=True)
state_unempl.to_csv('annual_unemployment_by_state.csv')

### State population data

In [38]:
state_pop = pd.DataFrame(columns = pd.DataFrame(fred.get_series('CAPOP')).T.columns)
for state in us_state_to_abbrev:
    id = us_state_to_abbrev[state]
    state_pop_df = pd.DataFrame(fred.get_series(id+'POP'))
    state_pop.loc[state] = state_pop_df.T.iloc[0]
state_pop.sort_index(axis=0, inplace=True)
state_pop.rename(columns=lambda x : str(x)[:4], inplace=True)
state_pop.drop([col for col in state_pop.columns if int(col) < start_year], axis=1, inplace=True)
state_pop.to_csv('population_by_state.csv')

### Personal consumption data

In [39]:
state_consumption = pd.DataFrame(columns = pd.DataFrame(fred.get_series('MIPCE')).T.columns)
for state in us_state_to_abbrev:
    id = us_state_to_abbrev[state]
    state_cons_df = pd.DataFrame(fred.get_series(id+'PCE'))
    state_consumption.loc[state] = state_cons_df.T.iloc[0]
state_consumption.sort_index(axis=0, inplace=True)
state_consumption.rename(columns=lambda x : str(x)[:4], inplace=True)
state_consumption.drop([col for col in state_consumption.columns if int(col) < start_year], axis=1, inplace=True)
state_consumption.to_csv('consumption_by_state.csv')

### Inflation data

In [67]:
pce_monthly = pd.DataFrame(fred.get_series('PCETRIM12M159SFRBDAL'))
pce_df = pce_monthly.T
pce_df = pce_df.filter(regex='[0-9]{4}-01-01')
for col in pce_df.columns:
    if col.year < start_year or col.year >= end_year:
        pce_df.drop(col, axis=1, inplace=True)
pce_df.rename(columns=lambda x : x.year, inplace=True)
pce_df.to_csv('annual_pce.csv')

pce_cumulative_df = pd.DataFrame(columns=pce_df.columns)
toAdd = [1]
for yr in range(start_year+1, end_year):
    toAdd.append(toAdd[-1] * (1 + pce_df[yr][0]/100))
pce_cumulative_df.loc['Cumulative PCE'] = toAdd
pce_cumulative_df.to_csv('cumulative_pce.csv')
